### Análisis del software de extracción de datos de ejecuciones workflow desde GitHub Actions

En esta celda se documenta el funcionamiento del script `script.py`, cuya finalidad es automatizar la recolección de información sobre las ejecuciones de GitHub Actions (*workflow runs*) de múltiples repositorios públicos.

#### Propósito general del software

Este software permite consultar la API de GitHub para obtener información detallada sobre la ejecución de workflows CI/CD de una lista de repositorios. Por cada ejecución encontrada, se descargan y almacenan:

1. Metadatos generales de la ejecución (`workflow_run.json`).
2. Detalles de los jobs que la componen (`jobs.json`).
3. El archivo YAML del workflow que define la ejecución.
4. Logs comprimidos (`logs.zip`).

Todo esto se guarda de forma organizada en carpetas por repositorio y ejecución, lo que permite crear un dateset para análisis posterior, comparaciones entre ejecuciones, estudios de fallos, etc.

#### ¿Cómo funciona el software?

1. **Lectura de repositorios:**
   - El script parte desde un archivo `repos.csv`, donde cada fila define un repositorio en formato `owner,repo`.
   - Se recorre esta lista para procesar los repositorios uno por uno.

2. **Conexión con la API de GitHub:**
   - Utiliza el token personal `GITHUB_TOKEN` (obtenido desde `.env`) para autenticar las peticiones.

3. **Extracción de workflow runs:**
   - Para cada repositorio se descargan hasta `MAX_RUNS` ejecuciones recientes.
   - Se consulta `/actions/runs` y se pagina si es necesario.

4. **Procesamiento por ejecución:**
   - Por cada ejecución el software realiza lo siguiente:
     - Se consulta su detalle con `/runs/{run_id}`.
     - Se obtienen los jobs asociados con `/runs/{run_id}/jobs`.
     - Se descarga el archivo YAML del workflow si es posible.
     - Se descargan los logs comprimidos.
   - Los archivos se guardan bajo una carpeta con nombre `run_id + nombre del workflow` sanitizado (el nombre debe ser sanitizado para evitar errores de ejecución por creación de ficheron con caracteres o largos no permitidos).

5. **Clasificación adicional:**
   - Si una ejecución terminó en **failure**, se copia también a la carpeta `failure_workflow_runs`.
   - Si es un **rerun** (es decir, `run_attempt > 1`), se copia además a `retry_workflow_runs`.

6. **Pausa controlada:**
   - El script incluye una espera (`time.sleep(1)`) entre ejecuciones para evitar llegar al límite de la API de GitHub y prevenir errores por throttling.

#### Organización de la información

La información se almacena bajo una carpeta por repositorio (con formato de nombre "owner_repo"), que a su vez contiene:

- `all_workflow_runs/`: Todas las ejecuciones.
- `failure_workflow_runs/`: Solo ejecuciones fallidas.
- `retry_workflow_runs/`: Solo reintentos.

Cada ejecución tiene su propia subcarpeta con todos los datos mencionados.

#### Notas importantes

- El script maneja errores de red y respuestas inválidas de la API con mensajes informativos y continúa con el siguiente repositorio o ejecución.
- Se sanitizan nombres de carpetas y archivos para evitar caracteres inválidos.
- Las funciones están organizadas modularmente para facilitar mantenimiento o reutilización.

La presente celda marca el punto previo a la ejecución del script del software. A continuación, ejecutaremos `script.py` para comenzar la recolección de datos.

In [ ]:
!python script.py